<a href="https://colab.research.google.com/github/Sakshibisen11/MovieReviews/blob/main/MovieReviews_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-09-12 18:13:03--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-09-12 18:13:03--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7e56b6b53c2124bdfafe80df39.dl.dropboxusercontent.com/cd/0/inline/Bs1EJjoV4iS_ZDZocB3xQ4tMBJWLPDYLHwiAW4EFHXTLO-lsoq7MuvnPPTJLsTDsrIhQOupiCuxHTUCaohGPnWXEORxjlaiQsUJ6cki54SpNy05KqeP83B7Dkw8ajTAKX48GEbiZ_vvgW4bCJUVb3OwNepoi3mt86OmiQ3jMVKJvLg/file# [following]
--2022-09-12 18:13:03--  https://uc7e56b6b53c2124bdfafe80df39.dl.dropboxusercontent.com/cd/0/inline/Bs1EJjoV4iS_ZDZocB3xQ4tMBJWLPDYLHwiAW4EFH

In [ ]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,RNN,SimpleRNNCell,Embedding,Dropout


In [ ]:
def prepareData(dir):
  data=text_dataset_from_directory(dir)
  return data.map(
      lambda text,label:(regex_replace(text,'<br/>',''),label),
  )

In [ ]:
train_data=prepareData('movie-reviews-dataset/train')
test_data=prepareData('movie-reviews-dataset/test')
for text_batch,label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'Hunky Geordie Robson Green is Owen Springer, a young doctor who moves home to Manchester to be near his father. Along the way, he falls for Anna, a woman 20 years his senior, and who happens to be the wife of his new boss, Richard Crane. Despite warnings from his new colleagues, Owen proceeds to get Anna for himself, going as far as to sabotage Anna and the cheating Richard\'s marriage. This is a romantic drama with many humorous undertones and a quick wit. The actors are superb: Green of "The Student Prince" and "Touching Evil" smolders on-screen as the cunning, yet warm-hearted Owen; Annis of "Dune" fame is lively and proves a good match to Green; Kitchen, from "To Play The King" is the right menace as Richard, whose comic missteps and snobbery underline his masterful, building hatred for Owen. This is a perfect love triangle, and despite the foibles and fallacies of our three characters, you come a

In [ ]:
model=Sequential()

In [ ]:
model.add(Input(shape=(1,),dtype="string"))

In [ ]:
#convert string input to vector representation
max_tokens=1000  #tokenize only 1000 words.Rest other words are considered as out of vocabulary
max_len=100 #determines maximum length of the vector
vectorize_layer=TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_len,
)

In [ ]:
#extract data from train_data object and train the vectorizor
train_texts=train_data.map(lambda text,label:text)
vectorize_layer.adapt(train_texts)
model.add(vectorize_layer)

In [ ]:
#convert integers into fixed length vectors
model.add(Embedding(max_tokens+1,128))
rnn=RNN(SimpleRNNCell(64),return_sequences=False,return_state=False)
#return_sequences=True while we want to add more RNN cells
#return_state=False for not viewing the hidden states
model.add(rnn)
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(train_data,epochs=1)

782/782 [==============================] - 18s 21ms/step - loss: 0.6947 - accuracy: 0.5176


In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 7s 9ms/step - loss: 0.6882 - accuracy: 0.5429


[0.688217282295227, 0.5429199934005737]

In [ ]:
text="I loved this movie"

In [ ]:
model.predict([text])

array([[0.51699096]], dtype=float32)